In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(0)

## Load Model

In [4]:
name = 'baseline_any_xse_resnext_512_appian'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)

In [7]:
m.avg_pool = nn.AdaptiveAvgPool2d(1)

In [8]:
# state['model'].keys()

In [9]:
# sp2 = '~/kaggle/rsna_retro/models/model100/fold0_ep2.pt'
sp3 = '~/kaggle/rsna_retro/models/model100/fold0_ep3.pt'

saved_path = Path(sp3).expanduser()
state = torch.load(saved_path)
m.load_state_dict(state['model'])

<All keys matched successfully>

In [10]:
dls = get_data(48, 512, splits=Meta.splits, img_dir=path_jpg)
learn = get_learner(dls, m)

## Training

In [11]:
do_fit(learn, 2, 5e-4, freeze=True)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.069960,0.074325,0.976008,0.955013,3:13:11


epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.070235,0.071104,0.976525,0.956204,3:12:03
1,0.054933,0.066092,0.978258,0.958980,3:11:02


## Submission

In [11]:
learn.load(f'runs/{name}-1')
sub_fn = f'subm/{name}'

In [ ]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')

In [ ]:
ob = learn.dls.one_batch()

In [ ]:
preds,targs = learn.get_preds()

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [16]:
api.competition_submit(f'{sub_fn}.csv', f'{name} appian se_resnext50', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.96MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [17]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13820147,
 'totalBytes': 27277209,
 'date': '2019-12-16T19:19:33.217Z',
 'description': 'baseline_any_xse_resnext_512_appian appian se_resnext50',
 'errorDescription': None,
 'fileName': 'baseline_any_xse_resnext_512_appian.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13820147/13820147.raw'}

## Save Predictions

In [9]:
path_appfeat512 = path/'appian_features_512'
path_appfeat512_tst = path/'appian_tst_features_512'

In [10]:
fn_save = f'runs/{name}-1'

In [11]:
dls = get_test_data(Meta.df_tst, bs=256, sz=None, tst_dir='tst_jpg')
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model.avg_pool = nn.AdaptiveAvgPool2d((4,4))
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512_tst)

In [14]:
dls = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg', sl=slice(300000,1000000))
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model.avg_pool = nn.AdaptiveAvgPool2d((4,4))
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512)

In [15]:
import gc
gc.collect()

3846